# Algorithms - Chapter 11 - Binary Search Trees

## 4/25/2023

### Joy Upton-Azzam

<mark>___________________________________________________________________________________________________________________________</mark>

#### 1. Binary Tree

In [1]:
%run "Alg_Ch_8(BinaryTree)-FOR_CH_11.ipynb"

<mark>___________________________________________________________________________________________________________________________</mark>

#### 3. Binary Search Tree

The following code implements the binary search tree from *Data Structures and Algorithms in Python* in chapter 11.

***Note***

* The print function from previous will not work
    * the updated get_coordinates is the issue with p.element() vs. str(p)
* There may be an issue with deleting the root item?
    * need to check the book to verify code/semantics

In [4]:
class TreeMap(BinaryTree,MapBase):
    """binary search tree based sorted map"""
    
    #----------------------------------------------------TreeMap Positions-------------------------------------------
    class Position(BinaryTree.Position):
        """position for tree-based maps"""
        
        def element(self):
            """returns the element for the position"""
            return self.node.element                             # converted to a string for printing
        
        def key(self):
            """returns the key for the node position"""
            return self.node.element.key                         # direct access via node.element
        
        def value(self):
            """returns the value for the node position"""
            return self.node.element.value                       # direct access via node.element
        
        def __str__(self):
            """returns a string of the Item for the position"""
            return "("+str(self.key())+","+str(self.value())+")"
        
    #----------------------------------------------------TreeMap Positions-------------------------------------------
    def _search(self,p,k):
        """returns the position of the item with key k (or the position of the closest node)"""
        current_key = p.key()    # key for item at position p
        
        if k == current_key:                                 # check for the key k
            return p
        elif k < current_key:
            if self.left(p):                                 # search left for smaller key k
                return self._search(self.left(p),k)
        elif self.right(p):                                  # search right for larger key k
                return self._search(self.right(p),k)
        return p                                             # this is the largest key <= k
    
    def first(self,p):
        """returns the position of the first item in the TreeMap"""
        while self.left(p) is not None:
            p = self.left(p)                           # move left until the end of the chain
        return p

    def after(self,p):
        """returns the position of the item with least key greater than or equal to p.key()"""
        if self.right(p) is not None:
            return self.first(self.right(p))           # find smallest key in right subtree
        else:
            parent = self.parent(p)                    # parent of current node
            while parent is not None and p == self.right(parent):
                p = parent                             # move up the tree until we find a left link or the root
                parent = self.parent(p)
            return parent                              # return ancestor with p in left subtree
    
    #----------------------------------------------------MAP FUNCTIONS-------------------------------------------
    def __getitem__(self,k):
        """returns the value for item with key k"""
        
        if self._size==0:
            raise KeyError('Key Error: '+repr(k))         # chcek for empty list
        else:
            p = self._search(self.root(),k)               # call the search function
            if p.key()!=k:                                # check for key
                raise KeyError('Key Error: '+repr(k))
            else:
                self._rebalance_access(p)#----------------# hook for re-balancing
                return p.value()                          # return value
    
    def __setitem__(self,k,v):
        """assign value v to key k"""
        if self._size == 0:
            self.add_root(MapBase.Item(k,v))              # add the first item
        else:
            p = self._search(self.root(),k)               # call the search function
            if p.key()==k:
                p.node.element.value = v                  # update value in list
                self._rebalance_access(p)#----------------# hook for re-balancing
            else:
                if p.key() < k:
                    p = self.add_right(p,MapBase.Item(k,v))   # add new node right of p
                else:
                    p = self.add_left(p,MapBase.Item(k,v))    # add new node left of p
                self._rebalance_insert(p)#--------------------# hook for re-balancing
    
    def __delitem__(self,k):
        """checks for item with key k and calls delete or raises an Error"""
        if self._size!=0:
            p = self._search(self.root(),k)            # position of node w/ largest key <= k
            if p.key()==k:                             # check for key
                self._delete(p)                        # call _delete on position p
                return
            self._rebalance_access(p)
        raise KeyError('Key Error: '+repr(k))
    
    def __iter__(self):
        """iterator for map"""
        p = self.first(self.root())        # start at the first position
        while p is not None:
            key = p.key()
            p = self.after(p)              # move to the position after p
            yield key
    
    #------------------------------------------------------------------------------------------------------------------
    def _delete(self,p):
        """deletes item at position p"""
        self.validate(p)
        k = p.key()
        parent = self.parent(p)

        if self.left(p) and self.right(p):    #-----------2 children----------------------------------
            r = self._search(self.left(p),k)  # position of largest key < k (largest in left sub-tree)
            item = r.node.element             # item to be moved
            self.__delitem__(r.key())         # call delitem (recursive)
            p.node.element = item             # replace deleted node p with r (effectively deleting p)
        
        elif parent:
            self.delete_attach(p)
        else:#--------------------deleting the root-------------------
            if self.left(p):#-------------Left child only-------------
                self._root = self._root.left            
                self._root.parent = None
            elif self.right(p):#----------right child only------------
                self._root = self._root.right
                self._root.parent = None
            else:#------------------------no children
                self._root = None
            self._size -= 1                   # decrease the size
    
    #------------------------------------------------------------------------------------------------------------------
    def _rebalance_access(self,p):
        """hook for re-balancing tree on access"""
        pass
    def _rebalance_insert(self,p):
        """hook for re-balancing tree on insertion"""
        pass
    def _rebalance_delete(self,p):
        """hook for re-balancing tree on deletion"""
        pass